In [3]:
import torch
import torch_directml

dml = torch_directml.device()  # gets the default DML device
x = torch.randn(1024, 1024, device=dml)
y = torch.mm(x, x)
print(dml, y.shape)


TypeError: 'staticmethod' object is not callable